# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns=['timestamp'])

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
# Define a Reader object
# The rating scale is specified; for the MovieLens dataset, ratings typically range from 0.5 to 5.0
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset from the DataFrame
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [6]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [7]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
# Define the parameter grid to search
param_grid = {
    'n_factors': [50, 100, 150],  # Number of factors
    'n_epochs': [20, 30],         # Number of iterations
    'lr_all': [0.005, 0.01],      # Learning rate for all parameters
    'reg_all': [0.02, 0.1]        # Regularization term
}

# Perform grid search using cross-validation
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)

# Fit the grid search
gs.fit(data)

# Output the best RMSE score and the corresponding parameters
print(f"Best RMSE score: {gs.best_score['rmse']}")
print(f"Best parameters: {gs.best_params['rmse']}")

Best RMSE score: 0.8550521122374983
Best parameters: {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1}


In [8]:
# print out optimal parameters for SVD after GridSearch
print(f"Best RMSE score: {gs.best_score['rmse']}")
print("Optimal parameters for SVD:")
for param, value in gs.best_params['rmse'].items():
    print(f"{param}: {value}")

Best RMSE score: 0.8550521122374983
Optimal parameters for SVD:
n_factors: 150
n_epochs: 30
lr_all: 0.01
reg_all: 0.1


In [9]:
# cross validating with KNNBasic
# Define the KNNBasic algorithm
knn_basic = KNNBasic()

# Perform cross-validation
cv_results = cross_validate(knn_basic, data, measures=['rmse'], cv=5, verbose=True)

# Output the RMSE results for each fold and the mean RMSE
print(f"Mean RMSE: {np.mean(cv_results['test_rmse'])}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9545  0.9445  0.9406  0.9469  0.9449  0.9463  0.0046  
Fit time          0.54    0.60    0.57    0.57    0.56    0.57    0.02    
Test time         4.56    4.47    4.59    4.59    4.47    4.54    0.05    
Mean RMSE: 0.9462696152137238


In [10]:
# print out the average RMSE score for the test set
# Perform cross-validation
cv_results = cross_validate(KNNBasic(), data, measures=['rmse'], cv=5, verbose=True)

# Calculate and print the average RMSE for the test set
average_rmse = np.mean(cv_results['test_rmse'])
print(f"Average RMSE score for the test set: {average_rmse}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9456  0.9523  0.9532  0.9456  0.9399  0.9473  0.0049  
Fit time          0.53    0.57    0.57    0.56    0.56    0.56    0.01    
Test time         4.46    4.45    4.64    4.68    4.63    4.57    0.10    
Average RMSE score for the test set: 0.9473126501578882


In [11]:
# cross validating with KNNBaseline
# Define the KNNBaseline algorithm
knn_baseline = KNNBaseline()

# Perform cross-validation
cv_results = cross_validate(knn_baseline, data, measures=['rmse'], cv=5, verbose=True)

# Calculate and print the average RMSE for the test set
average_rmse = np.mean(cv_results['test_rmse'])
print(f"Average RMSE score for KNNBaseline: {average_rmse}")

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8707  0.8779  0.8772  0.8752  0.8673  0.8737  0.0041  
Fit time          1.00    0.98    1.02    0.96    0.96    0.98    0.02    
Test time         6.47    6.37    6.42    6.57    6.54    6.48    0.07    
Average RMSE score for KNNBaseline: 0.8736821469926725


In [12]:
# print out the average score for the test set
# Perform cross-validation with KNNBaseline
cv_results = cross_validate(KNNBaseline(), data, measures=['rmse'], cv=5, verbose=True)

# Calculate and print the average RMSE for the test set
average_rmse = np.mean(cv_results['test_rmse'])
print(f"Average RMSE score for the test set: {average_rmse}")


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8665  0.8760  0.8673  0.8817  0.8743  0.8732  0.0057  
Fit time          1.07    0.94    1.09    0.99    0.95    1.01    0.06    
Test time         6.13    6.38    6.49    5.82    6.07    6.18    0.24    
Average RMSE score for the test set: 0.8731602061189893


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [13]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [14]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [15]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [16]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.064034527428012, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [17]:
def movie_rater(movie_df, num, genre=None):
    # Filter movies by genre if specified
    if genre:
        filtered_df = movie_df[movie_df['genres'].str.contains(genre, case=False)]
    else:
        filtered_df = movie_df
    
    # Randomly sample movies
    sampled_movies = filtered_df.sample(n=min(num, len(filtered_df)), random_state=1)
    
    rating_list = []
    
    # Collect ratings from the user
    for index, row in sampled_movies.iterrows():
        print(f"Movie: {row['title']}")
        try:
            rating = float(input("Rate this movie (1-5) or type 'skip' to skip: "))
            if rating < 1 or rating > 5:
                print("Rating should be between 1 and 5. Skipping...")
                continue
        except ValueError:
            print("Skipped rating.")
            continue
        
        # Append the rating to the list
        rating_list.append({
            'userId': 0,  # Assuming '0' for the new user. You can modify this as needed.
            'movieId': row['movieId'],
            'rating': rating
        })
    
    return rating_list
        

In [18]:
# try out the new function here!
# Assuming df_movies is your DataFrame with movie data
ratings = movie_rater(df_movies, num=5, genre='Comedy')
print(ratings)

Movie: Johnny Stecchino (1991)
Rate this movie (1-5) or type 'skip' to skip: 4
Movie: Filth (2013)
Rate this movie (1-5) or type 'skip' to skip: 3
Movie: Pain & Gain (2013)
Rate this movie (1-5) or type 'skip' to skip: 5
Movie: Cuckoo, The (Kukushka) (2002)
Rate this movie (1-5) or type 'skip' to skip: 2
Movie: Life-Size (2000)
Rate this movie (1-5) or type 'skip' to skip: 4
[{'userId': 0, 'movieId': 26732, 'rating': 4.0}, {'userId': 0, 'movieId': 105593, 'rating': 3.0}, {'userId': 0, 'movieId': 102033, 'rating': 5.0}, {'userId': 0, 'movieId': 6542, 'rating': 2.0}, {'userId': 0, 'movieId': 170837, 'rating': 4.0}]


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [22]:
## add the new ratings to the original ratings DataFrame
# Convert new ratings list to DataFrame
user_ratings_df = pd.DataFrame(ratings)

# Append new ratings to the original DataFrame
combined_df = pd.concat([df, user_ratings_df], ignore_index=True)

from surprise import Dataset, Reader

# Define the Reader with the rating scale
reader = Reader(rating_scale=(0.5, 5.0))

# Load the updated DataFrame into a Surprise dataset
data = Dataset.load_from_df(combined_df[['userId', 'movieId', 'rating']], reader)

In [24]:
# train a model using the new combined DataFrame
from surprise.model_selection import train_test_split

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Initialize and train the SVD model
svd = SVD(n_factors=50, reg_all=0.05)
svd.fit(trainset)

In [25]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
def get_user_predictions(user_id, model, movie_df, trainset):
    # Get the list of all movie IDs
    all_movie_ids = movie_df['movieId'].unique()
    
    # Get the list of movie IDs that the user has already rated
    rated_movie_ids = combined_df[combined_df['userId'] == user_id]['movieId'].unique()
    
    # Get the list of movie IDs that the user has not rated yet
    unrated_movie_ids = [mid for mid in all_movie_ids if mid not in rated_movie_ids]
    
    # Predict ratings for the unrated movies
    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]
    
    return predictions

In [26]:
# order the predictions from highest to lowest rated
def get_top_n_recommendations(predictions, n=10):
    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)
    
    # Extract the top N predictions
    top_n = sorted_predictions[:n]
    
    return top_n

# Get predictions for a specific user
user_id = 1  # Replace with the actual user ID
user_predictions = get_user_predictions(user_id, svd, df_movies, trainset)

# Get the top N recommendations
top_n_recommendations = get_top_n_recommendations(user_predictions, n=10)

# Print recommendations with movie titles
for pred in top_n_recommendations:
    movie_id = pred.iid
    movie_title = df_movies[df_movies['movieId'] == movie_id]['title'].values
    print(f"Movie ID: {movie_id}, Title: {movie_title[0] if len(movie_title) > 0 else 'Unknown'}, Predicted Rating: {pred.est}")

Movie ID: 318, Title: Shawshank Redemption, The (1994), Predicted Rating: 5.0
Movie ID: 750, Title: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964), Predicted Rating: 5.0
Movie ID: 912, Title: Casablanca (1942), Predicted Rating: 5.0
Movie ID: 1204, Title: Lawrence of Arabia (1962), Predicted Rating: 5.0
Movie ID: 1250, Title: Bridge on the River Kwai, The (1957), Predicted Rating: 5.0
Movie ID: 4973, Title: Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001), Predicted Rating: 5.0
Movie ID: 38061, Title: Kiss Kiss Bang Bang (2005), Predicted Rating: 5.0
Movie ID: 1237, Title: Seventh Seal, The (Sjunde inseglet, Det) (1957), Predicted Rating: 4.9880430603430534
Movie ID: 898, Title: Philadelphia Story, The (1940), Predicted Rating: 4.987476314505306
Movie ID: 1252, Title: Chinatown (1974), Predicted Rating: 4.984303868226268


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [49]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        pass
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  680    Philadelphia Story, The (1940)
Name: title, dtype: object 

Recommendation #  3 :  686    Rear Window (1954)
Name: title, dtype: object 

Recommendation #  4 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  5 :  926    Amadeus (1984)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.